In [ ]:
from matcher import name_preprocessing

In [ ]:
df = rpq('/mnt/da/patstat/patstat_parquet/tls906/',columns=['han_id','han_name','han_harmonized']
        ).query('han_harmonized>1')

In [ ]:
pandarallel.initialize(nb_workers=100)

In [ ]:
han = df.han_name.drop_duplicates().parallel_map(name_preprocessing).to_list()

In [ ]:
len(han)

In [ ]:
from rapidfuzz import process,fuzz

In [ ]:
choices = rpq('../ob_w_company_id_table_lm.pq',columns=['name_internat']
        ).name_internat.drop_duplicates().dropna().parallel_map(name_preprocessing).to_list()

In [ ]:
def run(name):
    return name,process.extract(name,choices,scorer=fuzz.WRatio,limit=100,score_cutoff=80)

with Pool(100) as p:
    with open('han_nm.csv','w') as f:
        wr = csv.writer(f)
        for n,res in tqdm(p.imap(run,han,chunksize=100),total=len(han)):
            for r in res:
                wr.writerow([n,res[0],int(res[1])])